In [ ]:
import os
import requests
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper

llm = ChatOpenAI(
    temperature = 0.1, 
    model_name="gpt-4.1-nano"
)

alpha_vantage_api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")

class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query:str = Field(description="The query you will search for")

class StockMarketSymbolSearchTool(BaseTool):
    name: Type[str] = "StockMarketSymbolSearchTool"
    args_schema : Type[
        StockMarketSymbolSearchToolArgsSchema
        ] = StockMarketSymbolSearchToolArgsSchema
    description: Type[str] = """
        Use this tool to find the stock market symbol for a company.
        It takes a query as an argument.
        Example query: Stock Market Symbol for Apple Inc
    """

    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)

class CompanyOverviewArgsSchema(BaseModel):

    symbol: str = Field(description="Stock symbol of the company.Example: AAPL, TSLA",)

class CompanyOverviewTool(BaseTool):
    name:Type[str] = "CompanyOverview"
    description: Type[str] = """
        Use this to get an overview of the financials of the company.
        You should enter a stock symbol.
    """
    args_schema : Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}")
        return r.json()
    

class CompanyOverviewArgsSchema(BaseModel):

    symbol: str = Field(
        description="Stock symbol of the company.Example: AAPL, TSLA",)


class CompanyIncomeStatementTool(BaseTool):
    name: Type[str] = "CompanyIncomeStatement"
    description: Type[str] = """
        Use this to get the income statement of a company.
        You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}")
        return r.json()["annualReports"]
    

class CompanyStockPerformanceTool(BaseTool):
    name: Type[str] = "CompanyStockPerformance"
    description: Type[str] = """
        Use this to get the weekly performance of a company stock.
        You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY&symbol={symbol}&apikey={alpha_vantage_api_key}")
        return r.json()


agent = initialize_agent(
    llm=llm,
    verbose = True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors = True,
    tools=[
        StockMarketSymbolSearchTool(),
        CompanyOverviewTool(),
        CompanyIncomeStatementTool(),
        CompanyStockPerformanceTool()
    ]
)

prompt = "Give me information on Cloudflare's stock, considering its financials, income statements and stock performance help me analyze if it's a potential good investment. Also tell me what symbol does the stock have."

agent.invoke(prompt)